**Subqueries That Return One Value**

What if we wanted to answer the question "Which books have more than the average number of version?". What is the questions within the question?

In [ ]:
SELECT AVG(books_count) AS avg_number_of_versions
FROM BooksDB.dbo.books

In [ ]:
SELECT 
    title,
    authors,
    books_count
FROM 
    BooksDB.dbo.books -- Not using an alias because we are using the same table in the subquery : no abiguity
WHERE 
    books_count > 75

In [ ]:
SELECT 
    title,
    authors,
    books_count
FROM 
    BooksDB.dbo.books -- Not using an alias because we are using the same table in the subquery : no abiguity
WHERE 
    books_count >
    (SELECT AVG(books_count)
     FROM BooksDB.dbo.books)

But why do we have to use a whole new internal query? Couldn't we just do:

In [ ]:
SELECT 
    title,
    authors,
    books_count
FROM 
    BooksDB.dbo.books AS b
WHERE 
    books_count > AVG(books_count)

Why can't you do this? Order of evaluations! The WHERE clause is applied to each row as it is read from the parent table to see if it should be included in the query evaluation, but aggregate functions are applied to all rows that pass the WHERE clause. You can't do them both at the same time

**Subqueries That Return Multiple Values**

What if we want to know the names of all of the tags that have been assigned to a certain book (e.g. book with book\_id=1)? What is the question within the question?

In [ ]:
SELECT 
    tag_id
FROM 
    BooksDB.dbo.book_tags
WHERE   
    goodreads_book_id = 1

In [ ]:
SELECT 
    t.tag_id, 
    t.tag_name
FROM 
    BooksDB.dbo.tags AS t 
WHERE 
    tag_id IN
    (SELECT tag_id
     FROM BooksDB.dbo.book_tags as bt 
     WHERE bt.goodreads_book_id = 1)
ORDER BY 
    t.tag_id ASC

Can we write this as a JOIN?

In [ ]:
SELECT 
    bt.goodreads_book_id,
    bt.tag_id,
    t.tag_name
FROM    
    BooksDB.dbo.book_tags AS bt
INNER JOIN
    BooksDB.dbo.tags AS t
ON  
    bt.tag_id = t.tag_id
WHERE   
    bt.goodreads_book_id = 1
ORDER BY
    bt.goodreads_book_id ASC,
    bt.tag_id ASC

Exercise Question 5: Write a query that returns the names of the tags and the tag ids for tags that were used over 100,000 times for a book. What is the question within the quesiton?

In [ ]:
SELECT DISTINCT(bt.tag_id)
FROM BooksDB.dbo.book_tags AS bt
WHERE count > 100000

In [ ]:
SELECT 
    t.tag_id,
    t.tag_name
FROM 
    BooksDB.dbo.tags AS t
WHERE t.tag_id IN
    (SELECT DISTINCT(bt.tag_id)
     FROM BooksDB.dbo.book_tags AS bt
     WHERE count > 100000)

Write a query that returns the names of the tags and the tags ids for tags that were used over 100,000 **in total across all books**

In [ ]:
SELECT 
    tag_id,
FROM 
    BooksDB.dbo.book_tags
GROUP BY
    tag_id
HAVING
    SUM(count) > 100000
ORDER BY 
    total_tags DESC

In [ ]:
SELECT 
    t.tag_id,
    t.tag_name
FROM 
    BooksDB.dbo.tags AS t
WHERE
    t.tag_id IN 
    (
        SELECT 
            bt.tag_id
        FROM 
            BooksDB.dbo.book_tags AS bt
        GROUP BY
            tag_id
        HAVING
            SUM(count) > 100000
    )

**Self-Contained Subqueries**

Everything we've done above!

**Correlated Subqueries**

Exercise Question 1: Write a query that will return the number of users who rated a book above its average rating. What is the question within the question? What is different about this 'question within a question'?

In [ ]:
SELECT 
    average_rating,
    book_id
FROM 
    BooksDB.dbo.books

In [ ]:
SELECT
    COUNT(*) AS happy_readers
FROM 
    BooksDB.dbo.ratings AS r
WHERE   
    r.rating > 
    (
        SELECT 
            b.average_rating
        FROM 
            BooksDB.dbo.books AS b
        WHERE
            b.book_id = r.book_id
    )

Write a query that will return the number of users for each particular book who rated the book above the average rating. What is the question within the question?

In [ ]:
SELECT
    r.book_id,
    COUNT(*) AS happy_readers
FROM 
    BooksDB.dbo.ratings AS r
WHERE   
    r.rating > 
    (
        SELECT 
            b.average_rating
        FROM 
            BooksDB.dbo.books AS b
        WHERE
            b.book_id = r.book_id
    )
GROUP BY
    r.book_id

**UNION**

Exercise Question 2: Write a query that returns the book ids of all books that have over 1000 ratings of 1 star OR over 1000 ratings of 5 stars

In [ ]:
SELECT
    book_id
FROM
    BooksDB.dbo.books
WHERE
    ratings_5 > 1000
UNION
SELECT
    book_id
FROM
    BooksDB.dbo.books
WHERE
    ratings_1 > 1000

**INTERSECT**

Exercise Question 3: Write a query that returns the book ids of all books that have over 1000 ratings of 1 star AND over 1000 ratings of 5 stars

In [ ]:
SELECT
    book_id
FROM
    BooksDB.dbo.books
WHERE
    ratings_5 > 1000
INTERSECT
SELECT
    book_id
FROM
    BooksDB.dbo.books
WHERE
    ratings_1 > 1000

**EXCEPT**

Exercise Question 4: Write a query that returns the book ids of books that have a language code of 'en-US' and not a language code of 'en-GB'

In [ ]:
SELECT 
    book_id
FROM 
    BooksDB.dbo.books
WHERE 
    language_code = 'en-US' 
    AND NOT
    language_code = 'en-GB'




SELECT 
    book_id
FROM 
    BooksDB.dbo.books
WHERE 
    language_code = 'en-US'
EXCEPT
SELECT 
    book_id
FROM 
    BooksDB.dbo.books
WHERE 
    language_code = 'en-GB'